In [1]:
import sys
import os
sys.path.append(os.path.abspath('../..'))

from utlis.sync_utlis.sync_df_utlis import *


In [2]:
base_folder = '/hpc/group/tdunn/lq53/sync_camera_/rawvid_mirstop_30s_light_3'

process_sync(base_folder, threshold=3, max_frames=100)

/hpc/group/tdunn/lq53/BBOP
